In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

# Step 1: Load the image
image = cv2.imread('fruits.jpg')  # Replace with your image path
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB (OpenCV loads in BGR by default)

# Step 2: Normalize the image
image_normalized = image / 255.0  # Scale pixel values to [0, 1] range

# Step 3: Reshape the image to a 2D array (each row is a pixel with RGB values)
pixels = image_normalized.reshape(-1, 3)  # Shape (num_pixels, 3)

# Step 4: Apply K-means clustering
k = 4  # Number of clusters
kmeans = KMeans(n_clusters=k, random_state=42)
kmeans.fit(pixels)  # Fit the model to the pixel data

# Step 5: Assign each pixel to the nearest cluster center
segmented_pixels = kmeans.cluster_centers_[kmeans.labels_]  # Replace each pixel with its centroid's RGB values

# Step 6: Reshape the segmented pixels back to the image shape
segmented_image = segmented_pixels.reshape(image.shape)

# Step 7: Visualize the results
plt.figure(figsize=(12, 6))

# Original Image
plt.subplot(1, 2, 1)
plt.title("Original Image")
plt.imshow(image)
plt.axis('off')

# Segmented Image
plt.subplot(1, 2, 2)
plt.title("Segmented Image (4 Clusters)")
plt.imshow(segmented_image)
plt.axis('off')

plt.show()


In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from skimage import color

# Step 1: Load the image
image = cv2.imread('fruits.jpg')  # Replace with your image path
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB (OpenCV loads in BGR by default)

# Step 2: Resize for faster processing (Optional)
image_resized = cv2.resize(image, (300, 300))  # Resize to a manageable size (Optional)

# Step 3: Normalize and convert to desired color space (Lab is recommended for color segmentation)
image_normalized = image_resized / 255.0  # Normalize to [0, 1]
image_lab = color.rgb2lab(image_normalized)  # Convert RGB to Lab color space

# Step 4: Reshape image to a 2D array of pixel values (for clustering)
pixels = image_lab.reshape(-1, 3)  # Flatten image to (num_pixels, 3)

# Step 5: Scale the pixel values (important for clustering)
scaler = StandardScaler()
pixels_scaled = scaler.fit_transform(pixels)

# Step 6: Use Elbow Method to determine optimal k (clusters)
wcss = []  # Within-cluster sum of squares
max_clusters = 10  # Maximum number of clusters to try

for k in range(1, max_clusters + 1):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(pixels_scaled)
    wcss.append(kmeans.inertia_)

# Plot Elbow Method to choose the optimal k
plt.figure(figsize=(8, 6))
plt.plot(range(1, max_clusters + 1), wcss, marker='o')
plt.title('Elbow Method for Optimal k')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('WCSS (Within-cluster sum of squares)')
plt.show()

# Step 7: Apply K-means clustering with the optimal k (for example, k=4)
optimal_k = 4  # Use the elbow method output to choose the best k

kmeans = KMeans(n_clusters=optimal_k, random_state=42)
kmeans.fit(pixels_scaled)
labels = kmeans.labels_

# Step 8: Create the segmented image using the cluster centers
segmented_pixels = kmeans.cluster_centers_[labels]  # Get the cluster centers for each pixel
segmented_pixels = scaler.inverse_transform(segmented_pixels)  # Inverse scaling to get back original values
segmented_pixels = segmented_pixels.reshape(image_resized.shape)  # Reshape back to original image shape

# Step 9: Convert back to RGB for display
segmented_image = color.lab2rgb(segmented_pixels)  # Convert back to RGB from Lab

# Step 10: Display results
plt.figure(figsize=(12, 6))

# Original Image
plt.subplot(1, 2, 1)
plt.title("Original Image")
plt.imshow(image_resized)
plt.axis('off')

# Segmented Image
plt.subplot(1, 2, 2)
plt.title("Segmented Image with K-Means (k=4)")
plt.imshow(segmented_image)
plt.axis('off')

plt.show()

